In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from multiprocessing import cpu_count
from multiprocessing import Pool
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import time
import re
from datetime import datetime
from multiprocessing import Pool
from multiprocessing import cpu_count

In [6]:
chrome_options = Options()
chrome_options.page_load_strategy = 'eager'  # Waits for minimal loading
driver = webdriver.Chrome(options=chrome_options)

# Set timeout and load URL
url = "https://economictimes.indiatimes.com/news/economy/policy"
driver.set_page_load_timeout(300)

try:
    driver.get(url)
    driver.execute_script("window.stop();")  # Stop loading after initial content
    print("Page loaded or stopped successfully.")
except Exception as e:
    print("Timeout or error occurred:", str(e))

Page loaded or stopped successfully.


In [7]:
#Function to click "Load More" button until it disappears or reaches the limit
def click_load_more(max_clicks=100):
    clicks = 0
    page_links = []
    while clicks < max_clicks:
        try:
            # Wait until the button is clickable
            load_more_button = WebDriverWait(driver, 30).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "autoload_continue"))
            )
            # driver.execute_script("window.scrollTo(0, arguments[0].getBoundingClientRect().top + window.pageYOffset);", load_more_button)
            total_height = driver.execute_script("return document.body.scrollHeight")
            # Calculate the scroll position with the offset
            scroll_position = total_height - 2000
            # Scroll to the calculated position
            driver.execute_script(f"window.scrollTo(0, {scroll_position});")
            time.sleep(2)
            # print(load_more_button)
            html_content = driver.page_source
            soup = BeautifulSoup(html_content, 'html5lib')
            pages = soup.select('div.eachStory>h3>a')
            # print("New:")
            # print(pages)
            for page in pages:
                link = page.get('href', 'No Link')
                if link != 'No Link' and link not in page_links:
                    page_links.append(link)
            ActionChains(driver).move_to_element(load_more_button).click(load_more_button).perform()
            clicks += 1
            print(f"Clicked 'Load More' {clicks} times")
            time.sleep(10)  # Allow time for content to load
        except Exception as e:
            print("No more 'Load More' button or an error occurred:", str(e))
            break
    return driver, page_links

# Click "Load More" button
driver1, page_links = click_load_more(max_clicks=100)
len(page_links)

Clicked 'Load More' 1 times
Clicked 'Load More' 2 times
Clicked 'Load More' 3 times
Clicked 'Load More' 4 times
Clicked 'Load More' 5 times
Clicked 'Load More' 6 times
Clicked 'Load More' 7 times
Clicked 'Load More' 8 times
Clicked 'Load More' 9 times
Clicked 'Load More' 10 times
Clicked 'Load More' 11 times
Clicked 'Load More' 12 times
Clicked 'Load More' 13 times
Clicked 'Load More' 14 times
Clicked 'Load More' 15 times
Clicked 'Load More' 16 times
Clicked 'Load More' 17 times
Clicked 'Load More' 18 times
Clicked 'Load More' 19 times
Clicked 'Load More' 20 times
Clicked 'Load More' 21 times
Clicked 'Load More' 22 times
Clicked 'Load More' 23 times
Clicked 'Load More' 24 times
Clicked 'Load More' 25 times
Clicked 'Load More' 26 times
Clicked 'Load More' 27 times
Clicked 'Load More' 28 times
Clicked 'Load More' 29 times
Clicked 'Load More' 30 times
Clicked 'Load More' 31 times
Clicked 'Load More' 32 times
Clicked 'Load More' 33 times
Clicked 'Load More' 34 times
Clicked 'Load More' 35 

1631

In [9]:
driver.quit()
page_links_final = page_links.copy()
for i, link in enumerate(page_links[:16]):
    page_links[i] = 'https://economictimes.indiatimes.com' + link
page_links

['https://economictimes.indiatimes.com/news/economy/policy/rbi-to-sell-rs-4-73-lakh-crore-government-bonds-in-march-quarter-for-states-and-uts/articleshow/116845280.cms',
 'https://economictimes.indiatimes.com/news/economy/policy/ilo-agrees-to-reassess-indias-social-protection/articleshow/116843434.cms',
 'https://economictimes.indiatimes.com/news/economy/policy/india-budget-as-we-enter-2025-a-lot-is-riding-on-sitharamans-feb-1-budget/articleshow/116825899.cms',
 'https://economictimes.indiatimes.com/news/economy/policy/25-for-2025-will-india-continue-to-wait-and-watch-is-the-big-crypto-question-on-everyones-mind/articleshow/116817393.cms',
 'https://economictimes.indiatimes.com/news/economy/policy/rbis-financial-stability-report-stress-may-tell-on-lenders-pile-on-npas-erode-buffers/articleshow/116808944.cms',
 'https://economictimes.indiatimes.com/news/economy/policy/credit-access-review-of-stressed-asset-norms-top-msme-bodys-wishlist/articleshow/116808677.cms',
 'https://economictime

In [13]:
def scrap(link):
    # link = page_links[1]
    description_page = requests.get(link)
    description = BeautifulSoup(description_page.content, 'html5lib')
    description_head = description.select('h1.artTitle')
    if len(description_head) == 1:
        description_head = description.select('h1.artTitle')[0].get_text(strip=True)
    date_str = description.select('time.jsdtTime')
    combined_datetime = ""
    if len(date_str) == 1:
        date_str = description.select('time.jsdtTime')[0].get_text(strip=True)
        match = re.search(r"(\w+ \d{1,2}, \d{4}), (\d{1,2}:\d{2}:\d{2} (AM|PM))", date_str)
        if match:
            date = match.group(1)
            time = match.group(2)
        else:
            date, time = "", ""
        combined_datetime = datetime.strptime(f"{date} {time}", "%b %d, %Y %I:%M:%S %p")
    summary = description.select('h2.summary')
    if len(summary) == 1:
        summary = description.select('h2.summary')[0].get_text(strip=True)
    description_content = description.select('div.artText')
    # print(description_content)
    # Initialize an empty list to store the filtered content
    filtered_text = []
    # Loop through each "artText m" div
    for div in description_content:
        # Stop processing if "div class=inSideInd show" is encountered
        if div.find_next('div', class_='inSideInd show'):
            break
        # Iterate over the children of the div
        for child in div.descendants:
            # Skip tags we want to exclude
            # print(child.name)
            if child.name == 'div' and 'inSideInd show' in child.get('class', []):
                break
            if any(parent.name in ['i', 'em', 'strong', 'style'] for parent in child.parents):
                continue
            # Handle <a> tags
            if child.name == 'a':
                # Include only if it has ref='dofollow'
                if child.get('rel') == ['dofollow']:
                    filtered_text.append(child.text.strip())
            # Include other text content directly
            elif child.name is None:  # It's a NavigableString
                filtered_text.append(child.text.strip())

    # Join the filtered text into a single string for output
    result = ' '.join(filtered_text)
    unwanted_portion_1 = "ET Year-end Special Reads 2024 Rewind: Elections, extreme weather, sporting glories, moments that made history, and heartfelt goodbyes Sensex & Nifty in 2025: Predictions, targets, must-have stocks for the new year From Adani bribery allegations to PayTM Bank ban: Six shocks that rocked India Inc in 2024 "
    unwanted_portion_2 = "Artificial Intelligence(AI) Java Programming with ChatGPT: Learn using Generative AI By - Metla Sudha Sekhar, IT Specialist and Developer View Program Artificial Intelligence(AI) Basics of Generative AI: Unveiling Tomorrows Innovations By - Metla Sudha Sekhar, IT Specialist and Developer View Program Artificial Intelligence(AI) Generative AI for Dynamic Java Web Applications with ChatGPT By - Metla Sudha Sekhar, IT Specialist and Developer View Program Artificial Intelligence(AI) Mastering C++ Fundamentals with Generative AI: A Hands-On By - Metla Sudha Sekhar, IT Specialist and Developer View Program Artificial Intelligence(AI) Master in Python Language Quickly Using the ChatGPT Open AI By - Metla Sudha Sekhar, IT Specialist and Developer View Program Marketing Performance Marketing for eCommerce Brands By - Zafer Mukeri, Founder- Inara Marketers View Program Office Productivity Zero to Hero in Microsoft Excel: Complete Excel guide 2024 By - Metla Sudha Sekhar, IT Specialist and Developer View Program Finance A2Z Of Money By - elearnmarkets, Financial Education by StockEdge View Program Marketing Modern Marketing Masterclass by Seth Godin By - Seth Godin, Former dot com Business Executive and Best Selling Author View Program Astrology Vastu Shastra Course By - Sachenkumar Rai, Vastu Shashtri View Program Strategy Succession Planning Masterclass By - Nigel Penny, Global Strategy Advisor: NSP Strategy Facilitation Ltd. View Program Data Science SQL for Data Science along with Data Analytics and Data Visualization By - Metla Sudha Sekhar, IT Specialist and Developer View Program Artificial Intelligence(AI) AI and Analytics based Business Strategy By - Tanusree De, Managing Director- Accenture Technology Lead, Trustworthy AI Center of Excellence: ATCI View Program Web Development A Comprehensive ASP.NET Core MVC 6 Project Guide for 2024 By - Metla Sudha Sekhar, IT Specialist and Developer View Program Marketing Digital Marketing Masterclass by Pam Moore By - Pam Moore, Digital Transformation and Social Media Expert View Program Artificial Intelligence(AI) AI-Powered Python Mastery with Tabnine: Boost Your Coding Skills By - Metla Sudha Sekhar, IT Specialist and Developer View Program Office Productivity Mastering Microsoft Office: Word, Excel, PowerPoint, and 365 By - Metla Sudha Sekhar, IT Specialist and Developer View Program Marketing Digital marketing - Wordpress Website Development By - Shraddha Somani, Digital Marketing Trainer, Consultant, Strategiest and Subject Matter expert View Program Office Productivity Mastering Google Sheets: Unleash the Power of Excel and Advance Analysis By - Metla Sudha Sekhar, IT Specialist and Developer View Program Web Development Mastering Full Stack Development: From Frontend to Backend Excellence By - Metla Sudha Sekhar, IT Specialist and Developer View Program Finance Financial Literacy i.e Lets Crack the Billionaire Code By - CA Rahul Gupta, CA with 10+ years of experience and Accounting Educator View Program Data Science SQL Server Bootcamp 2024: Transform from Beginner to Pro By - Metla Sudha Sekhar, IT Specialist and Developer View Program"
    result = result.replace(unwanted_portion_1, "").replace(unwanted_portion_2, "")

    return description_head, combined_datetime, summary, result

In [14]:
with Pool(cpu_count()) as p:
    results = p.map(scrap, page_links)

In [15]:
len(results)

1631

In [16]:
df = pd.DataFrame(results, columns=['Title', 'Timestamp', 'Synopsis', 'Content'])
df = df.replace('', np.nan)
df = df.dropna()
df

/tmp/ipykernel_47264/628576076.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace('', np.nan)


,Title,Timestamp,Synopsis,Content
0,RBI to sell Rs 4.73 lakh crore government bond...,2025-01-01 09:24:00,The Reserve Bank of India will sell government...,The Reserve Bank of India will sell government...
1,ILO agrees to reassess India's social protection,2025-01-01 06:56:00,The International Labour Organisation will rea...,The International Labour Organisation (ILO) ha...
2,"India Budget: As we enter 2025, a lot is ridin...",2024-12-31 14:42:00,Union Budget 2025: India's economy in 2025 hin...,"Budget 2025 : Entering the year 2025, India's ..."
3,#25 For 2025: Will India continue to ‘wait and...,2024-12-31 16:30:00,Crypto enthusiasts look for pro-industry laws ...,"With a ringing endorsement from Donald Trump, ..."
4,RBI’s financial stability report: Stress may t...,2024-12-31 05:30:00,"The report, published on Monday by the Reserve...",MUMBAI: The central bank’s Financial Stability...
...,...,...,...,...
1626,Rakesh Mohan on the World Bank Group’s Economi...,2024-04-04 18:08:00,"Ajay Banga, World Bank President, invited Rake...",President Emeritus and Distinguished Fellow of...
1627,India a bright spot in the Asia-Pacific region...,2024-04-05 00:01:00,"According to the Survey, released on Thursday,...",India has become the world’s fastest-growing m...
1628,What heat waves spell for India: Impact on RBI...,2024-04-04 15:16:00,Weather experts warn of increasing heat waves ...,"When one thinks of summers, one thinks of ice ..."
1629,Status quo for the 7th time? RBI MPC may not l...,2024-04-04 12:23:00,The RBI MPC is expected to maintain the repo r...,The Reserve Bank of India ( RBI )-led Monetary...


In [17]:
df.to_csv('/mnt/c/Users/ankku/Downloads/economic_times_policies.csv', index = False)